In [11]:
import math
import numpy as np
import os
import pandas as pd
import sys
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

def ZNorm(vals):
    vals = list(vals)
    vals = [float(x) for x in vals]
    m = np.mean(vals)
    sd = math.sqrt(np.var(vals))
    if sd == 0: return None
    if np.isnan(sd) or np.isnan(m): return None
    return [(item-m)*1.0/sd for item in vals]

def PROGRESS(msg, printit=True):
    if printit: # false for some messages when not in debug mode
        sys.stderr.write("%s\n"%msg.strip())

CHROM ='chr1'
DISTFROMGENE = 100000
path = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/Artery-Aorta//'
ESTR_RESULTS_FILE = path+'/HH//estr_result'
ESNP_RESULTS_FILE = path+'HH/esnp_result'
SNP_FDR_METHOD = 'qval.gene'
SNP_FDR_THRESHOLD = 1
STR_FDR_METHOD = 'qval.gene'
STR_FDR_THRESHOLD = 1
EXPRFILE = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/Artery-Aorta//Corr_Expr.csv'
EXPRANNOTFILE = '/storage/resources/dbase/human/hg19/gencode_gene_annotations_hg19.csv'
STRGTFILE = '/storage/szfeupe/Runs/650GTEx_estr/Genotypes/GenotypesNormalized.table'
SNPGTFILE = '/storage/szfeupe/Runs/650GTEx_estr/SNP_Analysis/'+str(CHROM)+'.tab'
NOHEAD = True
DEBUG = True

# Load expression and annotation
PROGRESS("Load expression", printit=DEBUG)
expr = pd.read_csv(EXPRFILE)
PROGRESS("Load annotation", printit=DEBUG)
expr_annot = pd.read_csv(EXPRANNOTFILE)
expr_annot.index = expr_annot["probe.id"].values
expr_annot = expr_annot[expr_annot["gene.chr"] == CHROM]
expr_annot = expr_annot.loc[[x for x in expr_annot.index if x in expr.columns],:]

# Load SNP genotypes
PROGRESS("Load SNPs", printit=DEBUG)
snpgt = pd.read_csv(SNPGTFILE, sep="\t", low_memory=False)
if CHROM: snpgt = snpgt[snpgt['chrom']==CHROM]
# Load STR genotypes
PROGRESS("Load STRs", printit=DEBUG)
strgt = pd.read_csv(STRGTFILE, sep="\t")
if CHROM: strgt = strgt[strgt['chrom']==CHROM]

# Restrict to samples with data in all three (SNP, STR, expression)
in_str_samples = set(strgt.columns[2:].values)
in_snp_samples = set(snpgt.columns[2:].values)
in_expr_samples = set(expr.index)
str_samples = list(in_str_samples.intersection(in_snp_samples).intersection(in_expr_samples))
print(len(str_samples), ' Samples')
expr = expr.loc[str_samples,:]
snpgt = snpgt[["chrom","start"] + str_samples]
strgt = strgt[["chrom","start"] + str_samples]

# Load eQTL results
PROGRESS("Load eQTL results", printit=DEBUG)
estr_results = pd.read_csv(ESTR_RESULTS_FILE, sep="\t")
esnp_results = pd.read_csv(ESNP_RESULTS_FILE, sep="\t")
if CHROM: 
    estr_results = estr_results[estr_results['chrom']==CHROM]
    esnp_results = esnp_results[esnp_results['chrom']==CHROM]

if DEBUG: print ('eqtl outputs loaded ', estr_results.shape, '\t',esnp_results.shape)
estr_results = estr_results[estr_results[STR_FDR_METHOD]<=STR_FDR_THRESHOLD]
esnp_results = esnp_results[esnp_results[SNP_FDR_METHOD]<=SNP_FDR_THRESHOLD]
if DEBUG: print ('eqtl threshold subeset ', estr_results.shape, '\t',esnp_results.shape)
if STR_FDR_METHOD == "qval.gene":
    estr_results = estr_results[estr_results["best_str"]==1]
    if DEBUG: print (estr_results.columns,'*******\n')
if DEBUG: print (esnp_results.shape )
if SNP_FDR_METHOD == "qval.gene":
    esnp_results = esnp_results[esnp_results["best_str"]==1]

allgenes = set(estr_results['gene'])   
print(len(allgenes))

Load expression
Load annotation
Load SNPs
Load STRs


191  Samples
eqtl outputs loaded  (164, 9) 	 (1750, 9)
eqtl threshold subeset  (164, 9) 	 (1750, 9)
Index(['gene', 'chrom', 'str.id', 'str.start', 'beta', 'best_str', 'qval.gene',
       'significant', 'pvalue'],
      dtype='object') *******

(1750, 9)
113


Load eQTL results


In [12]:
    count=0
    for ensgene in allgenes:
        count=count+1
        #print ensgene, esnp_results[esnp_results["gene"]==ensgene][SNP_FDR_METHOD]
        estr_fdr = min(estr_results[estr_results["gene"]==ensgene][STR_FDR_METHOD])
        esnp_fdr = min(esnp_results[esnp_results["gene"]==ensgene][SNP_FDR_METHOD])
        probes = expr_annot[expr_annot["gene.id"]==ensgene]["probe.id"].values
        if len(probes) != 1:
            PROGRESS("ERROR: no unique probe for for %s"%ensgene)
            continue
        else: probe = probes[0]
        PROGRESS("Getting data for %s (%s)"%(ensgene, probe), printit=DEBUG)
        # Get eSTRs
        estrs = estr_results[(estr_results["gene"]==ensgene)]
        # get eSNPs
        esnps = esnp_results[(esnp_results["gene"]==ensgene)]
        # Get genotypes and expression values
        y = pd.DataFrame({"expr":list(expr.loc[:,probe])})
        y.index = str_samples
        cis_snps = snpgt[snpgt["start"].apply(lambda x: x in list(esnps["str.start"].values))]
        cis_strs = strgt[strgt["start"].apply(lambda x: x in list(estrs["str.start"].values))]
        # Run one analysis per STR
        for i in range(cis_strs.shape[0]):
            # Get STR and SNP data
            locus_str = cis_strs.iloc[[i],:][str_samples].transpose()
            locus_str.index = str_samples
            locus_str.columns = ["STR_%s"%(cis_strs["start"].values[i])]
            locus_snp = cis_snps[str_samples].transpose()
            if len(locus_snp.columns) == 0:
                PROGRESS("Skipping %s, no SNP data."%ensgene, printit=DEBUG)
                continue
            # Get subsets
            samples_to_keep = [str_samples[k] for k in range(len(str_samples)) \
                               if (str(locus_str.iloc[:,0].values[k]) != "None" \
                                   and (str(locus_snp.iloc[:,0].values[k]) != "None"))]
            locus_str = locus_str.loc[samples_to_keep,:]

            locus_snp = locus_snp.loc[samples_to_keep,:]
            locus_y = y.loc[samples_to_keep,:]
            # Get data frame with relevant variables
            d = {"STR": ZNorm(locus_str.iloc[:,0].apply(float).values), "expr": ZNorm(locus_y["expr"].values)}
            for k in range(locus_snp.shape[1]):
                snps = ZNorm(locus_snp.iloc[:,k].values)
                if snps is not None:
                    d["SNP%s"%k] = snps
            genedata = pd.DataFrame(d)
            # Run regression with SNPs
            if locus_snp.shape[1] > 0 and "SNP0" in genedata.columns:
                snpstring = "+".join([item for item in list(genedata.columns) if "SNP" in item])
                formula_snpstr = "expr ~ STR + " + snpstring
                formula_snp = "expr ~ " + snpstring
                formula_str = "expr ~ STR"
                lm_snpstr = ols(formula_snpstr, genedata).fit()
                lm_snp = ols(formula_snp, genedata).fit()
                lm_str = ols(formula_str, genedata).fit()
                bic_snpstr = lm_snpstr.bic
                bic_snp = lm_snp.bic
                aic_snpstr = lm_snpstr.aic
                aic_snp = lm_snp.aic
                snpstr_rsq = lm_snpstr.rsquared
                snp_rsq = lm_snp.rsquared
                str_rsq = lm_str.rsquared
                anova_results = anova_lm(lm_snp, lm_snpstr)
                pval = anova_results["Pr(>F)"].values[1]
                print (",".join(map(str, [CHROM, ensgene, cis_strs["start"].values[i], locus_snp.shape[1],\
                                              locus_y.shape[0], str_rsq, snp_rsq, snpstr_rsq, pval, estr_fdr, esnp_fdr,\
                                             bic_snp-bic_snpstr, aic_snp-aic_snpstr])))




Getting data for ENSG00000086619.9 (ENSG00000086619.9)
/usr/lib64/python3.4/site-packages/scipy/stats/_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/usr/lib64/python3.4/site-packages/scipy/stats/_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/usr/lib64/python3.4/site-packages/scipy/stats/_distn_infrastructure.py:1814: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
Getting data for ENSG00000116260.12 (ENSG00000116260.12)


chr1,ENSG00000086619.9,236423126,1,182,0.08865420519920142,0.09482019697042454,0.10868477392812581,0.09693577878046479,0.00288768904969689,0.11366316901899401,-2.3947552670222194,0.8092514200545793
chr1,ENSG00000086619.9,236455432,1,159,0.10645980835953472,0.06688066480472965,0.1268068816216814,0.0013147086078055888,0.00288768904969689,0.11366316901899401,5.484936046294649,8.55384024851486
chr1,ENSG00000116260.12,180119587,1,183,0.026490903345970596,0.1704904783923853,0.18226315607188437,0.10919776837413193,0.0509356303043826,0.00100686143312287,-2.593682296998793,0.6158038558426142


Getting data for ENSG00000142632.12 (ENSG00000142632.12)


chr1,ENSG00000142632.12,16541915,1,182,0.2173457728389483,0.20161592046743437,0.22137688184955995,0.034419811708887346,0.0017977677683353199,0.00100686143312287,-0.6426029947750749,2.561403692301724
chr1,ENSG00000142632.12,16597735,1,167,0.1367207312229567,0.17844131798042406,0.19895312972803114,0.04203076580979212,0.0017977677683353199,0.00100686143312287,-0.8955812362193001,2.222412576197428


Getting data for ENSG00000215915.5 (ENSG00000215915.5)


...

chr1,ENSG00000198160.10,67460327,1,175,0.06933366612510017,0.0708557406826471,0.08612564348436258,0.09183354115235297,0.03160357824149,0.929498559573085,-2.2648758855538063,0.8999100883696656


Getting data for ENSG00000143156.9 (ENSG00000143156.9)


chr1,ENSG00000143156.9,169166646,1,190,0.1394490714777551,0.15196267774790306,0.17598430215981042,0.020613894861546345,0.0017977677683353199,0.00100686143312287,0.21263854539733984,3.4596626175577967
chr1,ENSG00000143156.9,169260885,1,177,0.1652054268553267,0.18531771795321506,0.2228313488091661,0.004237674908647041,0.0017977677683353199,0.00100686143312287,3.1677750782858425,6.343924810859676
chr1,ENSG00000143156.9,169288257,1,176,0.1845089520805946,0.1390640559960613,0.20580314401909672,0.0001908453414538474,0.0017977677683353199,0.00100686143312287,9.03073503475514,12.201219029793265


Getting data for ENSG00000168528.7 (ENSG00000168528.7)


chr1,ENSG00000168528.7,31886001,1,182,0.12607571237961335,0.06419977799765308,0.13654764144496612,0.00015073342286342186,0.0017977677683353199,0.9384848811741279,9.440312473289623,12.644319160366422


In [10]:
count

,gene,chrom,str.id,str.start,beta,best_str,qval.gene,significant,pvalue
2327,ENSG00000070371.11,chr22,STR_19229814,19229814,0.196814,1,0.023884,NaN,5.580001e-04
2328,ENSG00000070371.11,chr22,STR_19295540,19295540,-0.235948,1,0.001983,1.0,1.723015e-05
2329,ENSG00000073146.11,chr22,STR_50571464,50571464,0.183377,1,0.058313,0.0,1.915390e-03
2330,ENSG00000075240.12,chr22,STR_46947159,46947159,0.183776,1,0.055394,0.0,1.769302e-03
2331,ENSG00000093000.14,chr22,STR_45524668,45524668,-0.194372,1,0.057318,0.0,1.856737e-03
2332,ENSG00000099917.13,chr22,STR_20892992,20892992,-0.164047,1,0.081954,0.0,3.154200e-03
2340,ENSG00000099940.7,chr22,STR_21313548,21313548,-0.169058,1,0.074006,NaN,2.694442e-03
2339,ENSG00000099940.7,chr22,STR_21304617,21304617,-0.206626,1,0.016298,NaN,3.374964e-04
2338,ENSG00000099940.7,chr22,STR_21301012,21301012,-0.212880,1,0.021311,NaN,4.726935e-04
2337,ENSG00000099940.7,chr22,STR_21298693,21298693,0.205772,1,0.009535,NaN,1.670719e-04


In [7]:
ensgene

'ENSG00000100196.6'

'STR_31054363'

In [ ]:


for ensgene in allgenes:
    count=count+1
    print (ensgene, 'Starting...')#,esnp_results[esnp_results["gene"]==ensgene][SNP_FDR_METHOD])
    estr_fdr = min(estr_results[estr_results["gene"]==ensgene][STR_FDR_METHOD])
    esnp_fdr = min(esnp_results[esnp_results["gene"]==ensgene][SNP_FDR_METHOD])
    #print(ensgene, estr_fdr.shape, esnp_fdr.shape)
    probes = expr_annot[expr_annot["gene.id"]==ensgene]["probe.id"].values
    if len(probes) != 1:
        PROGRESS("ERROR: no unique probe for %s"%ensgene, printit=DEBUG)
        continue
    else: probe = probes[0]
    
    if esnp_fdr==[] or estr_fdr==[]:
        PROGRESS("ERROR: qvalue for %s was not calculated for one these (SNPs or STRs)"%ensgene, printit=DEBUG)
        continue
    
    PROGRESS("Getting data for %s (%s)"%(ensgene, probe), printit=DEBUG)
    # Get eSTRs # get eSNPs
    estrs = estr_results[(estr_results["gene"]==ensgene)]
    esnps = esnp_results[(esnp_results["gene"]==ensgene)]
    esnps = esnps.sort_values(SNP_FDR_METHOD).head(10)  #Taking only top 10 snps
    # Get genotypes and expression values
    y = pd.DataFrame({"expr":list(expr.loc[:,probe])})
    y.index = str_samples
    cis_snps = snpgt[snpgt["start"].apply(lambda x: x in list(esnps["str.start"].values))]
    cis_strs = strgt[strgt["start"].apply(lambda x: x in list(estrs["str.start"].values))]
    print(cis_strs.shape, cis_snps.shape, '-------------cis variants---------------')
    # Run one analysis per STR
    for i in range(cis_strs.shape[0]):
        # Get STR and SNP data
        locus_str = cis_strs.iloc[[i],:][str_samples].transpose()
        locus_str.index = str_samples
        locus_str.columns = ["STR_%s"%(cis_strs["start"].values[i])]
        locus_snp = cis_snps[str_samples].transpose()
        print ("STR Locus ", i, '\t ', locus_str.shape, locus_snp.shape)
        if len(locus_snp.columns) == 0:
            PROGRESS("Skipping %s, no SNP data."%ensgene, printit=DEBUG)
            continue
        # Get subsets
        samples_to_keep = [str_samples[k] for k in range(len(str_samples)) \
                           if (str(locus_str.iloc[:,0].values[k]) != "None" \
                               and (str(locus_snp.iloc[:,0].values[k]) != "None"))]
        locus_str = locus_str.loc[samples_to_keep,:].replace('None', np.nan).astype(float)
        locus_snp = locus_snp.loc[samples_to_keep,:].replace('None', np.nan).astype(float)
        locus_y = y.loc[samples_to_keep,:]
        
        # Get data frame with relevant variables
        d = {"STR": ZNorm(locus_str.iloc[:,0].values), "expr": ZNorm(locus_y["expr"].values)}
        for k in range(locus_snp.shape[1]):
            snps = ZNorm(locus_snp.iloc[:,k].values)
            if snps is not None:
                d["SNP%s"%k] = snps
        genedata = pd.DataFrame(d)
        print(genedata.shape, '::::::::::::::::::::::::')
        # Run regression with SNPs
        if locus_snp.shape[1] > 0 and "SNP0" in genedata.columns:
            snpstring = "+".join([item for item in list(genedata.columns) if "SNP" in item])
            formula_snpstr = "expr ~ STR + " + snpstring
            formula_snp = "expr ~ " + snpstring
            formula_str = "expr ~ STR"
            lm_snpstr = ols(formula_snpstr, genedata).fit()
            lm_snp = ols(formula_snp, genedata).fit()
            lm_str = ols(formula_str, genedata).fit()
            bic_snpstr = lm_snpstr.bic
            bic_snp = lm_snp.bic
            aic_snpstr = lm_snpstr.aic
            aic_snp = lm_snp.aic
            snpstr_rsq = lm_snpstr.rsquared
            snp_rsq = lm_snp.rsquared
            str_rsq = lm_str.rsquared
            anova_results = anova_lm(lm_snp, lm_snpstr)
            pval = anova_results["Pr(>F)"].values[1]
            print (",".join(map(str, [CHROM, ensgene, cis_strs["start"].values[i], locus_snp.shape[1],locus_y.shape[0], \
                                           str_rsq, snp_rsq,snpstr_rsq, pval, estr_fdr, esnp_fdr,bic_snp-bic_snpstr, aic_snp-aic_snpstr])))
        else:
            formula_str = "expr ~ STR"
            lm_str = ols(formula_str, genedata).fit()
            str_rsq = lm_str.rsquared
            print (",".join(map(str, [CHROM, ensgene, cis_strs["start"].values[i], locus_snp.shape[1],\
                                          locus_y.shape[0], str_rsq, 0, str_rsq, -1, estr_fdr, -1, 0, 0])))
    